In [2]:
import pandas             as pd
import numpy              as np
import matplotlib.pyplot  as plt
from plyer import notification 
from datetime import datetime
import requests
import os
import zipfile

In [3]:
def baixar_e_extrair(url, nome_arquivo):
  
  # Verifica se o arquivo já existe
  if os.path.exists(nome_arquivo):
    print(f"O arquivo {nome_arquivo} já existe. Pulando o download.")
    return

  # Baixa o arquivo
  response = requests.get(url)
  response.raise_for_status()

  # Salva o arquivo no disco
  with open(nome_arquivo, 'wb') as f:
    f.write(response.content)

  # Extrai o arquivo ZIP
  with zipfile.ZipFile(nome_arquivo, 'r') as zip_ref:
    zip_ref.extractall()

# URL do arquivo ZIP:
url = "https://download.inep.gov.br/microdados/microdados_saeb_2021_ensino_fundamental_e_medio.zip"  
nome_arquivo = "microdados_saeb_2021_ensino_fundamental_e_medio.zip"  # Nome do arquivo baixado

# Chama a função para baixar e extrair o arquivo
baixar_e_extrair(url, nome_arquivo)

O arquivo microdados_saeb_2021_ensino_fundamental_e_medio.zip já existe. Pulando o download.


In [4]:
def alerta(nivel, base, etapa, erro=""):
    now = str(datetime.now())
    msg = f"Falha no carregamento da base {base} na etapa {etapa}.\n{now}\n{erro}"

    if nivel == 1:
        title = 'ATENÇÃO: Alerta Baixo'
    elif nivel == 2:
        title = 'ATENÇÃO: Alerta Médio'
    elif nivel == 3:
        title = 'ATENÇÃO: Alerta Alto'
    else:
        print("Nível", nivel, "não disponível!")
        return

    # Enviar notificação
    notification.notify(
        title=title,
        message=msg,
        app_name='alerta',
        timeout=10
    )

In [5]:
def carregar_csv(caminho_csv):
    return pd.read_csv(caminho_csv, encoding='latin1', sep=';')

caminho_csv = "DADOS/TS_ESCOLA.csv"
df = carregar_csv(caminho_csv)
df.head(5)


,ID_SAEB,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_ESCOLA,IN_PUBLICA,ID_LOCALIZACAO,PC_FORMACAO_DOCENTE_INICIAL,PC_FORMACAO_DOCENTE_FINAL,...,MEDIA_5EF_LP,MEDIA_5EF_MT,MEDIA_9EF_LP,MEDIA_9EF_MT,MEDIA_EMT_LP,MEDIA_EMT_MT,MEDIA_EMI_LP,MEDIA_EMI_MT,MEDIA_EM_LP,MEDIA_EM_MT
0,2021,1,11,6316600,2,61324549,1,1,100.0,51.1,...,157.99,168.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021,1,11,6316600,2,61335087,1,1,92.5,63.9,...,NaN,NaN,255.49,254.43,NaN,NaN,NaN,NaN,NaN,NaN
2,2021,1,11,6316600,2,61336597,1,2,100.0,56.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021,1,11,6316600,2,61342611,1,1,100.0,53.6,...,206.92,216.53,NaN,NaN,250.12,246.73,NaN,NaN,250.12,246.73
4,2021,1,11,6316600,2,61363380,1,1,87.5,26.9,...,170.73,169.64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
